In [1]:
from selenium import webdriver
from selenium.webdriver.support.wait import WebDriverWait
import time
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.action_chains import ActionChains

In [2]:
ruc_prueba = "20153219118"

In [3]:
driver = webdriver.Chrome()
driver.maximize_window()
driver.get("https://prod2.seace.gob.pe/seacebus-uiwd-pub/buscadorPublico/buscadorPublico.xhtml")
wait = WebDriverWait(driver, 20)
time.sleep(5)

#### NOMBRE O SIGLA DE ENTIDAD

In [4]:
lupa_buscar_entidad = wait.until(EC.presence_of_element_located((By.ID, "tbBuscador:idFormBuscarProceso:j_idt28"))).click()
time.sleep(1)

In [5]:
elemento_input = driver.find_element(By.CSS_SELECTOR,'[id="tbBuscador:idFormBuscarProceso:txtRucEntidad"][name="tbBuscador:idFormBuscarProceso:txtRucEntidad"]').click()
ActionChains(driver).send_keys_to_element(elemento_input, ruc_prueba).perform()

In [6]:
btn_buscar = driver.find_element(By.ID, "tbBuscador:idFormBuscarProceso:btnBuscarEntidad").click()
time.sleep(1)

In [7]:
op_confirmar = driver.find_element(By.ID, "tbBuscador:idFormBuscarProceso:dataTable:0:ajaxEntidad").click()          

In [8]:
print("LISTO NOMBRE O SIGLA DE ENTIDAD")

LISTO NOMBRE O SIGLA DE ENTIDAD


#### OBJETO DE CONTRATACIÓN

In [9]:
desp_contratacion = wait.until(EC.presence_of_element_located((By.ID, 'tbBuscador:idFormBuscarProceso:j_idt41_label'))).click()
time.sleep(1)

In [10]:
elemento_servicio = driver.find_element(By.CSS_SELECTOR, 'li[data-label="Servicio"]').click()

In [11]:
print("LISTO OBJETO DE CONTRATACIÓN")

LISTO OBJETO DE CONTRATACIÓN


#### AÑO DE LA CONVOCATORIA

In [12]:
desp_convocatoria = wait.until(EC.presence_of_element_located((By.ID, "tbBuscador:idFormBuscarProceso:anioConvocatoria_label"))).click()
time.sleep(1)

In [13]:
elemento_2023 = driver.find_element(By.CSS_SELECTOR, 'li[data-label="2023"]').click()

In [14]:
print("LISTO OBJETO DE CONTRATACIÓN")

LISTO OBJETO DE CONTRATACIÓN


#### CÓDIGO CAPTCHA

In [15]:
from datetime import datetime

In [16]:
def fn_get_imagen():
    imagen = driver.find_element(By.CSS_SELECTOR, 'img[id="tbBuscador:idFormBuscarProceso:captchaImg"]')
    x, y, width, height = imagen.location['x'], imagen.location['y'], imagen.size['width'], imagen.size['height']
    imagen_pil = Image.open(io.BytesIO(driver.get_screenshot_as_png()))
    imagen_recortada = imagen_pil.crop((x, y, x + width, y + height))
    nombre = "captcha" + datetime.now().strftime("%d_%m_%Y-%H_%M_%S") + ".png"
    imagen_recortada.save(nombre)    
    time.sleep(2)
    return imagen_recortada

In [17]:
def fn_lectura_ocr(image):
    grayscale_image = image.convert('L')
    threshold = 50  
    binary_image = grayscale_image.point(lambda p: p < threshold and 255)
    text = pytesseract.image_to_string(binary_image)
    text = text.lower()
    text = text.replace('\n', '')
    text = text.replace("'",'')
    text = text.replace(" ","")
    return text

In [18]:
#devuelve un valor no vacío
def fn_evaluacion(i):
    
    imagen_recortada = fn_get_imagen()
    time.sleep(2)
    
    lectura_ocr = fn_lectura_ocr(imagen_recortada)
    time.sleep(2)
    print("-------------------------------")
    print("Lectura_ocr_",i,": ",lectura_ocr)
    time.sleep(2)
    
    while len(lectura_ocr) != 5:
        
        print ("Lectura incorrecta, nuevo intento...")
        i = i + 1
        
        elemento_captcha = driver.find_element(By.CSS_SELECTOR,'button[id="tbBuscador:idFormBuscarProceso:btnrefreshcaptcha"][name="tbBuscador:idFormBuscarProceso:btnrefreshcaptcha"]').click()
        time.sleep(2)
        
        imagen_recortada = fn_get_imagen()
        time.sleep(2)

        lectura_ocr = fn_lectura_ocr(imagen_recortada)
        time.sleep(2)
        
        print("-------------------------------")
        print("Lectura_ocr_",i,": ",lectura_ocr)
        time.sleep(2)
        
    return lectura_ocr,i

In [19]:
from PIL import Image
import io

In [20]:
#pip install pytesseract
import os

# Ruta al directorio tessdata
tessdata_dir = "C:\\Program Files\\Tesseract-OCR\\tessdata"

# Configurar la variable de entorno TESSDATA_PREFIX
os.environ["TESSDATA_PREFIX"] = tessdata_dir

In [21]:
import pytesseract
pytesseract.pytesseract.tesseract_cmd = r'C:\Program Files\Tesseract-OCR\tesseract.exe'

In [22]:
from selenium.common.exceptions import NoSuchElementException

In [23]:
i=1

In [24]:
#ver si hubo error
elemento = 'x'

while elemento:
    rpta,i = fn_evaluacion(i)
    time.sleep(2)
    
    print("ubicar campo para captcha...")
    input_captcha = driver.find_element(By.CSS_SELECTOR,'[id="tbBuscador:idFormBuscarProceso:codigoCaptcha"][name="tbBuscador:idFormBuscarProceso:codigoCaptcha"][type="text"]').click()
    ActionChains(driver).send_keys(rpta).perform()
    time.sleep(2)
    
    print("buscando info...")
    elemento_buscar = driver.find_element(By.CSS_SELECTOR,'[id="tbBuscador:idFormBuscarProceso:btnBuscarSel"][name="tbBuscador:idFormBuscarProceso:btnBuscarSel"]').click()
    time.sleep(1)
    
    try:
        elemento = driver.find_element(By.CSS_SELECTOR, '[class="ui-growl-message"]')
    except NoSuchElementException:
        print("BÚSQUEDA EJECUTADA CORRECTAMENTE")
        break
    print("Lectura incorrecta, nuevo intento...")
    time.sleep(1)


-------------------------------
Lectura_ocr_ 1 :  r766f
ubicar campo para captcha...
buscando info...
BÚSQUEDA EJECUTADA CORRECTAMENTE


In [ ]:
CONTINUAR CON... recoleccion de resultados
Me imagino que habrán criterios de los que le interesen, consultarlo a Cesar
